## Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:


In [ ]:
!docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [3]:
!curl localhost:9200

{
  "name" : "b74c19275d98",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "pqtPQeYoQAubspIHBU_Lrg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


What's the `version.build_hash` value?


## Getting the data

Now let's get the FAQ data. You can run this snippet:

In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Note that you need to have the `requests` library:

```bash
pip install requests
```

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

In [6]:
!pipenv install elasticsearch

Installing elasticsearch...
Resolving elasticsearch...
✔ Installation Succeeded
⠋ Installing elasticsearch...
Installing dependencies from Pipfile.lock (9e3cfe)...


In [8]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200")

In [9]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course'])

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [12]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc) 

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [21]:
query="How do I execute a command in a running docker container?"

search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

first_hit = response['hits']['hits'][0]
first_hit["_score"]


84.050095

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [22]:
query = "How do I execute a command in a running docker container?"
query_filter = "machine-learning-zoomcamp"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": query_filter
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    print(hit['_source']['question'])


How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)

In [23]:
context_template = """
Q: {question}
A: {text}
""".strip()

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

In [24]:

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [31]:
context = ""
for hit in response['hits']['hits']:
    context = context + context_template.format(
        question=hit["_source"]["question"],
        text=hit["_source"]["text"]
    ) + "\n\n"
context = context.strip()
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [32]:
prompt = prompt_template.format(question=query, context=context)
len(prompt)

1462

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

In [33]:
!pipenv install tiktoken

Installing tiktoken...
Resolving tiktoken...
Added tiktoken to Pipfile's [packages] ...
✔ Installation Succeededen...
⠋ Installing tiktoken...
Pipfile.lock (9e3cfe) out of date, updating to (2a1e40)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠸ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (2d25fe3ed45894dc8b13eaf611cd2b964bdb3db1a4ea0f740d0e7692fb2a1e40)!
Installing dependencies from Pipfile.lock (2a1e40)...


Let's calculate the number of tokens in our query: 

In [40]:
import tiktoken

encoding_gpt4 = tiktoken.encoding_for_model("gpt-4o")
tokens_gpt4 = encoding_gpt4.encode(prompt)

In [41]:
len(tokens_gpt4)

322

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:


encoding.decode_single_token_bytes(63842)


## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

In [45]:
!pipenv install ollama

Installing ollama...
Resolving ollama...
Added ollama to Pipfile's [packages] ...
✔ Installation Succeeded...
⠋ Installing ollama...
Pipfile.lock (2a1e40) out of date, updating to (ace5f8)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠴ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (4be5891bef6e77fafcece701cd75bc3e79c43bb4f28ce3d94a25da6aedace5f8)!
Installing dependencies from Pipfile.lock (ace5f8)...


In [46]:
import ollama
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])
print(response['message']['content'])

According to the FAQ database context, to execute a command in a running Docker container:

docker exec -it <container-id> bash

Replace `<container-id>` with the actual ID of your running container.


## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

- Input: $0.005 / 1K tokens

- Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [44]:
price = 150*0.005+250*0.015
print("$", price, "per 150k input, 250k output")

$ 4.5 per 150k input, 250k output


In [50]:
tokens_gpt4_output = encoding_gpt4.encode(response['message']['content'])
len(tokens_gpt4_output)

41

In [55]:
price = len(tokens_gpt4)*0.005+len(tokens_gpt4_output)*0.015
print("$", price, "per", len(tokens_gpt4),"k input, ", len(tokens_gpt4_output), "k output")

$ 2.225 per 322 k input,  41 k output


## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw1
* It's possible that your answers won't match exactly. If it's the case, select the closest one.